# Retrieve Data
---

In this chapter we will explore different ways to retrieve the [ECMWF real-time open data](./find-data.ipynb) using Python libraries. The main focus of this handbook will be on the `earthkit` and `ecmwf-opendata` packages.

:::{figure} ../images/earthkit.svg
Source: https://github.com/ecmwf/earthkit
:::

## The development environment setup
The tutorials use the [Python](https://www.python.org/) programming language and its libraries:
- [`earthkit`](https://earthkit.readthedocs.io/en/latest/) to speed up weather and climate science workflows
- [`ecmwf-opendata`](https://github.com/ecmwf/ecmwf-opendata) to download the ECMWF open data
- [`requests`](https://pypi.org/project/requests/) (or [`wget`](https://www.gnu.org/software/wget/manual/wget.html)) for sending HTTP requests
- [`xarray`](https://pypi.org/project/xarray/) for multi-dimensional arrays of geospatial data
- [`pandas`](https://pypi.org/project/pandas/) to perform powerful operations on datasets
- [`matplotlib`](https://pypi.org/project/matplotlib/) for creating static and interactive visualizations
- [`cartopy`](https://pypi.org/project/Cartopy/) for cartographic visualizations
- [`plotly`](https://pypi.org/project/plotly/) for interactive data visualization
- [`geopandas`](https://pypi.org/project/geopandas/) to handle geographic data of pandas objects

We will install our packages using the `!pip3 install <package name>` commands. The Jupyter Notebooks will execute these as shell commands as at the beginning of each command is the mark `!`.

:::{tip}
In case one do not use a [virtual environment](https://docs.python.org/3.12/library/venv.html) to install Python packages within it, it is recommended using the `--user` option for `pip3 install` to ensure that packages are installed in the current user account's location and therefore separate from other users. It is important when multiple users share a device.
:::

### 1. The `earthkit` and `ecmwf-opendata` package

Here we will retrieve ECMWF real-time open data from **ECMWF Data Store (ECPDS)**. <br>

If the packages are not installed yet, uncomment the code below and run it.

In [ ]:
# !pip3 install earthkit-data ecmwf-opendata requests datetime

In [2]:
from ecmwf.opendata import Client
import earthkit.data as ekd

In [5]:
client = Client(source="ecmwf")
request = {
    "date" : -1,
    "time" : 0,
    "step" : 12,
    "type" : "fc",
    "stream": "oper",
    "levtype" : "sfc",
    "model" : "aifs-single",
    "param" : "2t",
}
client.retrieve(request, "2t.grib2")

ds_2t = ekd.from_source("file", "2t.grib2")
ds_2t.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,2t,heightAboveGround,2,20250517,0,12,fc,None,regular_ll


In [8]:
ds_uv = ekd.from_source("ecmwf-open-data",
                        time=12,
                        param=["u", "v"],
                        levelist=[1000, 850, 500],
                        step=0
                       )
ds_uv.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,u,isobaricInhPa,1000,20250517,1200,0,fc,None,regular_ll
1,ecmf,u,isobaricInhPa,850,20250517,1200,0,fc,None,regular_ll
2,ecmf,u,isobaricInhPa,500,20250517,1200,0,fc,None,regular_ll
3,ecmf,v,isobaricInhPa,1000,20250517,1200,0,fc,None,regular_ll
4,ecmf,v,isobaricInhPa,850,20250517,1200,0,fc,None,regular_ll
5,ecmf,v,isobaricInhPa,500,20250517,1200,0,fc,None,regular_ll


:::{hint}
When you want to download data for more models, e.g. `model=["ifs", "aifs-single"]`), you can list them and then filter the data based on the criteria you specify.
:::

In [4]:
data = ekd.from_source("ecmwf-open-data",
                       date=-1,
                       time=12,
                       step=0,
                       param=['msl', 'tp'],
                       stream="oper",
                       type="fc",
                       levtype="sfc",
                       model=["ifs", "aifs-single"]
                       )
data.describe()

,,level,date,time,step,number,paramId,class,stream,type,experimentVersionNumber
shortName,typeOfLevel,,,,,,,,,,
msl,meanSea,0,20250821,1200,0,None,151,"ai,od",oper,fc,0001
tp,surface,0,20250821,1200,0,None,"228228,228","ai,od",oper,fc,0001


### 2. The `requests` package

In [3]:
import requests
import datetime

DATADIR = './'
today = datetime.date.today().strftime('%Y%m%d') # a user can choose current date or data up to four days before today 
timez = "00z/"
model = "aifs-single/"
resol = "0p25/"
stream_ = "oper"
type_ = "fc"
step = "6"
filename = f'{today}{timez[:-2]}0000-{step}h-{stream_}-{type_}.grib2'

with requests.Session() as s:
    start = datetime.datetime.now()
    response = requests.get(f'https://data.ecmwf.int/ecpds/home/opendata/{today}/{timez}{model}{resol}{stream_}/{filename}', stream=True)
    response.raise_for_status()
    with open(filename, mode="wb") as file:
        for chunk in response.iter_content(chunk_size=10 * 1024):
            file.write(chunk)
    end = datetime.datetime.now()
    diff = end - start
    print(f'The {filename} file downloaded in {diff.seconds} seconds.')

data = ekd.from_source("file", f'{DATADIR}/{filename}')
data.ls()

The 20250827000000-6h-oper-fc.grib2 file downloaded in 7 seconds.


,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,mcc,mediumCloudLayer,800,20250827,0,6,fc,None,regular_ll
1,ecmf,vsw,soilLayer,1,20250827,0,6,fc,None,regular_ll
2,ecmf,vsw,soilLayer,2,20250827,0,6,fc,None,regular_ll
3,ecmf,z,isobaricInhPa,400,20250827,0,6,fc,None,regular_ll
4,ecmf,q,isobaricInhPa,925,20250827,0,6,fc,None,regular_ll
...,...,...,...,...,...,...,...,...,...,...
97,ecmf,z,isobaricInhPa,600,20250827,0,6,fc,None,regular_ll
98,ecmf,w,isobaricInhPa,300,20250827,0,6,fc,None,regular_ll
99,ecmf,u,isobaricInhPa,925,20250827,0,6,fc,None,regular_ll
100,ecmf,t,isobaricInhPa,600,20250827,0,6,fc,None,regular_ll


### 3. The `wget` command-line tool

To install `wget` on Linux, execute
```
sudo apt-get install wget
```
For extremely large files, it is recommended to use the `-b` option that will download your content in the background. In your working directory a `wget-log` will also appear that can be used to check your download progress and status. You can save the file you retrieve in another directory using the `-P` option
```
ROOT="https://data.ecmwf.int/forecasts"
yyyymmdd="20250525"
HH="00"
model="ifs"
resol="0p25"
stream="oper"
step="24"
U="h"
type="fc"
format="grib2"
wget -P ../datadownload/ -b "$ROOT/$yyyymmdd/$HH"z"/$model/$resol/$stream/$yyyymmdd$HH"0000"-$step$U-$stream-$type.$format"
```

:::{note}
For more information, see [Access real-time open data with `wget`](https://confluence.ecmwf.int/display/DAC/ECMWF+open+data%3A+real-time+forecasts+from+IFS+and+AIFS#ECMWFopendata:realtimeforecastsfromIFSandAIFS-Locations).
:::

### 4. The [`curl`](https://curl.se/) command-line tool
When you need to download a single field from a GRIB file, inspect the corresponding index file and look for the parameter of your interest. For example, to download only the 2 m temperature at step=0h from the 00 UTC HRES forecast on 08 June 2025
```
{"domain": "g", "date": "20250608", "time": "0000", "expver": "0001", "class": "od", "type": "fc", "stream": "oper", "step": "0", "levtype": "sfc", "param": "2t", "_offset": 62556419, "_length": 660091}
```
use the values of `_offset`  and `_length` keys and calculate the `start_bytes` and `end_bytes`

:::{important}
The `_offset` and `_length` values of a specific field are different for each forecast run!
:::

```
start_bytes = _offset = 62556419
end_bytes = _offset + _length - 1 = 62556419 + 660091 - 1 = 63216509
```

```
ROOT="https://data.ecmwf.int/forecasts"
yyyymmdd="20250608"
HH="00"
model="ifs"
resol="0p25"
stream="oper"
step="0"
U="h"
type="fc"
format="grib2"
start_bytes=62556419
end_bytes=63216509

curl --range "$start_bytes-$end_bytes" "$ROOT/$yyyymmdd/$HH"z"/$model/$resol/$stream/$yyyymmdd$HH"0000"-$step$U-$stream-$type.$format" --output 2t.grib2
```